In [1]:
import bs4
import requests
import os
import re
import os
import datetime
import json

In [2]:
# download main page to get download links
page = requests.get('https://catalog.unc.edu/courses/')
f = open('/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/courses.html', "wb")
f.write(page.content)
f.close()

In [3]:
# get links to all course pages
file = open('/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/courses.html', 'r')
html = bs4.BeautifulSoup(file, 'html5lib')
file.close()

categories = []
links = html.find('ul', {'id': '/courses/'}).find_all('a')
links = list(links)
for link in links:
  categories.append(link['href'])

categoryLinks = []
for category in categories:
  categoryLinks.append(f'https://catalog.unc.edu{category}')

categoryLinks

['https://catalog.unc.edu/courses/aero/',
 'https://catalog.unc.edu/courses/aaad/',
 'https://catalog.unc.edu/courses/amst/',
 'https://catalog.unc.edu/courses/anth/',
 'https://catalog.unc.edu/courses/appl/',
 'https://catalog.unc.edu/courses/arab/',
 'https://catalog.unc.edu/courses/arch/',
 'https://catalog.unc.edu/courses/army/',
 'https://catalog.unc.edu/courses/arth/',
 'https://catalog.unc.edu/courses/asia/',
 'https://catalog.unc.edu/courses/astr/',
 'https://catalog.unc.edu/courses/bioc/',
 'https://catalog.unc.edu/courses/bcb/',
 'https://catalog.unc.edu/courses/bbsp/',
 'https://catalog.unc.edu/courses/biol/',
 'https://catalog.unc.edu/courses/bmme/',
 'https://catalog.unc.edu/courses/bios/',
 'https://catalog.unc.edu/courses/bcs/',
 'https://catalog.unc.edu/courses/busi/',
 'https://catalog.unc.edu/courses/chip/',
 'https://catalog.unc.edu/courses/cata/',
 'https://catalog.unc.edu/courses/cbph/',
 'https://catalog.unc.edu/courses/cbio/',
 'https://catalog.unc.edu/courses/cb

In [4]:
if len(categoryLinks) == len(categories):
  for i in range(len(categories)):
    page = requests.get(categoryLinks[i])
    f = open(f'/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/{categories[i][:-1]}.html', "wb")
    f.write(page.content)
    f.close()

In [8]:
directory = '/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/courses'
output = []
for file in os.listdir(directory):
  f = os.path.join(directory, file)
  if os.path.isfile(f):
    print(f'\n{datetime.datetime.now()} current file: {file}')
    page = open(f, 'r')
    html = bs4.BeautifulSoup(page, 'html5lib')
    page.close()
    courses = list(html.findAll('div', {'class': 'courseblock'}))
    for course in courses:
      try:
        rawTitle = course.findAll('p', {'class': 'courseblocktitle'})
        rawTitle = rawTitle[0].stripped_strings
        for title in rawTitle:
          # get raw title text and clean it
          title = repr(title)
          if 'xa0' in title:
            title = title.replace('\\xa0', ' ')
          title = re.sub(r'(\\n)', ' ', title)
          title = re.sub('\s+', ' ', title)
          title = title.split('.')

          # get course code
          courseCode = title[0].strip()

          # get course title
          courseTitle = title[1].strip()

          # get number of credit hours awarded
          creditHoursAwarded = title[2].strip()
        
        # get raw text of description and clean it
        rawDesc = course.findAll('p', {'class': 'courseblockdesc'})
        rawDesc = rawDesc[0].get_text()
        rawDesc = re.sub(r'(\\n)', ' ', rawDesc)
        rawDesc = re.sub('\s+', ' ', rawDesc)

        # get course description
        try:
          if 'Gen Ed:' in rawDesc:
            courseDesc = re.split(r'Gen Ed:', rawDesc)
            courseDesc = courseDesc[0].strip()
        except:
          print(f'no course description found for {courseCode}')
          courseDesc = 'none'

        # get making connections requirement info
        try:
          if 'Repeat rules:' in rawDesc:
            makingConnections = re.split(r'Gen Ed:(.+?\.)', rawDesc)
            makingConnections = makingConnections[1].strip()
        except:
          print(f'no making connections information found for {courseCode}')
          makingConnections = 'none'

        # get repeat rules:
        try:
          if 'Repeat rules:' in rawDesc:
            repeatRules = re.split(r'Repeat rules:(.+?)Grading', rawDesc)
            repeatRules = repeatRules[1].strip()
        except:
          print(f'no repeat rules found for {courseCode}')
          repeatRules = 'none'
      
        # get grading status rules 
        try:
          if 'Grading status:' in rawDesc:
            gradingStatus = re.split(r'Grading status:(.+?\.)', rawDesc)
            gradingStatus = gradingStatus[1].strip()
        except:
          print(f'no repeat rules found for {courseCode}')
          gradingStatus = 'none'

        courseCode = {
          'category': file,
          'course code': courseCode,
          'course title': courseTitle,
          'credit hours awarded': creditHoursAwarded,
          'description': courseDesc,
          'making connections requirement': makingConnections,
          'ideas in action requirement': '',
          'repeat rules': repeatRules,
          'grading status': gradingStatus,
        }
        print(courseCode)
        # output.append(courseCode)
      except:
        print(f'an unknown error occured with {file} at {datetime.datetime.now()}')






2022-05-10 13:41:31.290991 current file: arch.html
{'category': 'arch.html', 'course code': "'ARCH 393", 'course title': 'Internship in Archaeology', 'credit hours awarded': '3-6 Credits', 'description': 'Permission of the instructor and the director of undergraduate studies. Internships combine substantive work experience with an academic project.', 'making connections requirement': 'EE- Academic Internship.', 'ideas in action requirement': '', 'repeat rules': 'May be repeated for credit. 12 total credits. 2 total completions.', 'grading status': 'Letter grade.'}
no making connections information found for 'ARCH 395
{'category': 'arch.html', 'course code': "'ARCH 395", 'course title': 'Research in Archaeology', 'credit hours awarded': '1-6 Credits', 'description': 'Permission of the instructor and the director of undergraduate studies. Internships combine substantive work experience with an academic project.', 'making connections requirement': 'none', 'ideas in action requirement': '

KeyboardInterrupt: 